In [ ]:
import numpy as np
import keras
from keras import backend as k
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
Inception_resnet_V2 = keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=None,input_shape=(224,224,3), pooling= 'avg', classes=1000)

In [ ]:
train_path = 'F:/Master/Data/Training' 
valid_path = 'F:/Master/Data/Validation'
test_path = 'F:/Master/Data/Test'

In [ ]:
train_batches = ImageDataGenerator().flow_from_directory(train_path,target_size=(224,224),classes=['cable','no'],batch_size = 10)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path,target_size=(224,224),classes=['cable','no'],batch_size = 10)
test_batches = ImageDataGenerator().flow_from_directory(test_path,target_size=(224,224),classes=['cable','no'],batch_size = 10,shuffle = False)

In [ ]:
x = Inception_resnet_V2.layers[-1].output
predictions = Dense(2,activation = 'softmax')(x)
model = Model(inputs=Inception_resnet_V2.input,outputs=predictions)

In [ ]:
model.summary()

In [ ]:
for layer in model.layers [:-2]:
    layer.trainable = False

In [ ]:
model.compile(Adam(lr=.00001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit_generator(train_batches,steps_per_epoch=140,validation_data = valid_batches,validation_steps=40,epochs=30,verbose=2)

In [ ]:
test_labels = test_batches.classes

In [ ]:
predictions = model.predict_generator(test_batches,steps=20,verbose=0)

In [ ]:
cm = confusion_matrix(test_labels,predictions.argmax(axis=1))

In [ ]:
test_batches.class_indices

In [ ]:
#Evaluation of Model - Confusion Matrix Plot
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


In [ ]:
cm_plot_labels = ['Cable','No Cable']
plot_confusion_matrix(cm,cm_plot_labels,title= 'Confusion Matrix')

In [ ]:
model.save('classification_of_cable_with_Inception_resnet_V2.h5')